In [1]:
!pip install biopython

ERROR: Could not find a version that satisfies the requirement biopython (from versions: none)
ERROR: No matching distribution found for biopython


# **1.1 Loading datasets and data filtration**

In [2]:
import pandas as pd
import os 

# Listing files in the dataset
os.listdir('/kaggle/input/cancergenelist/')

# Loading the data from a loaded dataset
oncokb_df = pd.read_csv("/kaggle/input/cancergenelist/cancerGeneList.tsv", sep="\t")

print("Number of genes before filtration: ", len(oncokb_df))

# Show first rows to chceck if the format is correct
oncokb_df.head()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Number of genes before filtration:  1192


,Hugo Symbol,Entrez Gene ID,GRCh37 Isoform,GRCh37 RefSeq,GRCh38 Isoform,GRCh38 RefSeq,Is Oncogene,Is Tumor Suppressor Gene,# of occurrence within resources (Column J-P),OncoKB Annotated,MSK-IMPACT,MSK-HEME,FOUNDATION ONE,FOUNDATION ONE HEME,Vogelstein,COSMIC CGC (v99),Gene Aliases
0,ABL1,25,ENST00000318560,NM_005157.4,ENST00000318560,NM_005157.4,Yes,No,7,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"ABL, JTK7, c-ABL"
1,AKT1,207,ENST00000349310,NM_001014431.1,ENST00000349310,NM_001014431.1,Yes,No,7,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"AKT, PKB, PRKBA, RAC, RAC-alpha"
2,ALK,238,ENST00000389048,NM_004304.4,ENST00000389048,NM_004304.4,Yes,No,7,Yes,Yes,Yes,Yes,Yes,Yes,Yes,CD246
3,AMER1,139285,ENST00000330258,NM_152424.3,ENST00000374869,NM_152424.3,No,Yes,7,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"FAM123B, FLJ39827, RP11-403E24.2, WTX"
4,APC,324,ENST00000257430,NM_000038.5,ENST00000257430,NM_000038.5,No,Yes,7,Yes,Yes,Yes,Yes,Yes,Yes,Yes,"DP2.5, PPP1R46"


In [3]:
# Filtrating loaded genes by Vogelstein
vogelstein_df = oncokb_df[oncokb_df['Vogelstein'] == 'Yes']
# Saving names of the filtrated genes into a variable
vogelstein_genes = vogelstein_df['Hugo Symbol'].tolist()
# Changing for string to SQL
genes_str = ",".join(f"'{gene}'" for gene in vogelstein_genes)

print("Number of genes after filtration: ", len(vogelstein_genes))
print(genes_str)

Number of genes after filtration:  125
'ABL1','AKT1','ALK','AMER1','APC','AR','ARID1A','ASXL1','ATM','ATRX','AXIN1','BAP1','BCL2','BCOR','BRAF','BRCA1','BRCA2','CARD11','CBL','CDC73','CDH1','CDKN2A','CEBPA','CIC','CREBBP','CTNNB1','DAXX','DNMT3A','EGFR','EP300','ERBB2','EZH2','FBXW7','FGFR2','FGFR3','FLT3','FOXL2','GATA3','GNA11','GNAQ','GNAS','HNF1A','HRAS','IDH1','IDH2','JAK1','JAK2','JAK3','KDM5C','KDM6A','KIT','KMT2D','KRAS','MAP2K1','MAP3K1','MED12','MEN1','MET','MLH1','MPL','MSH2','MSH6','MYD88','NF1','NF2','NFE2L2','NOTCH1','NOTCH2','NPM1','NRAS','PAX5','PBRM1','PDGFRA','PIK3CA','PIK3R1','PPP2R1A','PRDM1','PTCH1','PTEN','PTPN11','RB1','RET','RNF43','SETD2','SF3B1','SMAD2','SMAD4','SMARCA4','SMARCB1','SMO','SOCS1','SPOP','STAG2','STK11','TET2','TNFAIP3','TP53','TSC1','U2AF1','VHL','WT1','ARID2','B2M','CASP8','CRLF2','CSF1R','FUBP1','GATA1','GATA2','H3-3A','H3C2','KMT2C','PHF6','RUNX1','SRSF2','ARID1B','CYLD','KLF4','NCOR1','SETBP1','SOX9','TRAF7','TSHR','ACVR1B','DNMT1'


# **1.2** Mutation downloading

In [4]:
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient

# Setting BigQuery client
client = bigquery.Client()

# SQL: Dowloading mutations only for filtrated genes
query = f"""
SELECT *
FROM `isb-cgc-bq.TCGA.masked_somatic_mutation_hg38_gdc_current`
WHERE Hugo_Symbol IN ({genes_str})
AND Variant_Type IN ('SNP', 'DEL', 'INS')
"""

query_results = client.query(query)
mutations_df = query_results.to_dataframe()

print("Found records of filtrated genes: ", len(mutations_df))


Using Kaggle's public dataset BigQuery integration.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Found records of filtrated genes:  48912


# **1.3** Parquet format

In [7]:
# Saving filtrated records to Parquet
mutations_df.to_parquet('filtered_mutations.parquet')

In [8]:
# Downloading transcriptomic data from TCGA
query = f"""SELECT DISTINCT *
FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current`
WHERE gene_name in ({genes_str});
"""
query_job = client.query(query)
df_TCGA_rna = query_job.to_dataframe()

In [ ]:
#and saving to parquet
df_TCGA_rna.to_parquet('expression.parquet')

# **1.4** Primary sites characteristics

In [ ]:
print(df_TCGA_rna.columns)
#how many recodrs in expression df
print(len(df_TCGA_rna))

In [ ]:
#checking how many records are for each primary site
print(df_TCGA_rna['primary_site'].value_counts())

# **Zad 2 features ingeneering**

In [19]:
#I had a problem dowloading this file here by kaggle. Chat said i could be a DNS problem
# so I uploaded it to input manually
!wget http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

--2025-06-28 11:40:20--  http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘hgdownload.soe.ucsc.edu’


In [5]:
!gunzip "/kaggle/input/hg38_dataset/hg38.fa.gz"

gzip: /kaggle/input/hg38_dataset/hg38.fa.gz: No such file or directory


In [ ]:
import Bio
print(Bio.__version__)


In [1]:
!pip install biopython

ERROR: Could not find a version that satisfies the requirement biopython (from versions: none)
ERROR: No matching distribution found for biopython


In [7]:
import Bio
from Bio import SeqIO

fasta_sequences = SeqIO.to_dict(SeqIO.parse("/kaggle/input/hg38_dataset/hg38.fa", "fasta"))


ModuleNotFoundError: No module named 'Bio'

In [10]:
def get_motif(row):
    ref = row['Reference_Allele']
    alt = row['Tumor_Seq_Allele2']
    context = row['context']  # np. ACA, TTC itp.
    return f"{context[0]}[{ref}>{alt}]{context[2]}"

mutations_df['motif'] = mutations_df.apply(get_motif, axis=1)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


KeyError: 'context'